<a href="https://colab.research.google.com/github/wandb/davis-contest/blob/main/colabs/final_submit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![header](https://i.imgur.com/sAPM7Yy.png)

# Submitting Final Results for DAVIS Contest

Starting at midnight Pacific time on March 29th,
the test data (without labels) for the DAVIS contest will be made available.
In order to make a submission to the contest for final evaluation,
you will need to apply your model to that data
and upload the outputs within 72 hours -- by 11:59 pm Pacific time on March 31st.

This notebook includes instructions and example code for this submission process,
demonstrated on the `DummyModel` trained in the
[PyTorch starter notebook](https://wandb.me/davis-starter-pt).



**End-to-end, the submission process is**:
1. Download the test data from W&B
1. Apply your model to the test data and upload results to W&B
1. Submit your results at the [new leaderboard](https://wandb.ai/wandb/davis-contest-test/benchmark/leaderboard).

**NOTE**: Before the test set is made available,
the data downloaded by the code below
is a _randomized_ version of the actual test set.
This dataset has the same structure (folder names, file sizes, etc.)
as the real test set, but the image files contain random bits.
This allows you to check your submission pipeline for bugs and issues before the
test data is released.
Once the test data is released, the same code will download that data instead.

In [ ]:
%%capture
!pip install wandb==0.10.23
!pip install git+https://github.com/wandb/davis-contest#egg=contest[torch,keras]
!apt install tree

In [ ]:
import wandb

import contest
from contest.utils import paths

# Download Test Data

In [ ]:
!wandb login --relogin

The test data,
consisting of over 2000 frames across 22 video clips,
is provided as a W&B Artifact,
just like the training and validation data.

These cells download that test data from [wandb.ai](https://wandb.ai).

Note that only an obfuscated/randomized version of the test data
is available before the test phase begins.

Once the test phase is begins, the exact same code will
download the correct version of the test data.

In [ ]:
test_data_artifact_name = "/".join(["wandb", "davis", "davis-contest-test"])
# once the real test set is released, the "latest" tag will point to it
tag = "latest"

test_data_artifact_id = test_data_artifact_name + ":" + tag

In [ ]:
with wandb.init(project="davis", job_type="download_data") as run:
  test_data_artifact = run.use_artifact(test_data_artifact_id)
  test_data_path = test_data_artifact.download()

# Apply Model to Data and Upload Results to W&B

As during the earlier phase of the contest,
results are submitted by
1. generating model outputs and logging them to a W&B run, then
2. submitting that run to a W&B Benchmark.

Unlike during that phase,
participants don't have access to the ground truth segmentations,
and so won't be submitting their scores directly.
Instead, results will be scored against a private ground truth.
This helps ensure that the actual generalization performance of the model
is being assessed.

In [ ]:
####
## REPLACE THIS CELL WITH YOUR MODEL SETUP CODE
####

import torch
import pytorch_lightning as pl


class DummyModel(pl.LightningModule):

  def __init__(self):
    super().__init__()
    self.conv = torch.nn.Conv2d(in_channels=3, out_channels=1, kernel_size=1)

  def forward(self, xs):
    return torch.sigmoid(self.conv(xs))

  def training_step(self, batch, batch_idx):
    loss = self.forward_on_batch(batch)
    return loss

  def validation_step(self, batch, batch_idx):
    loss = self.forward_on_batch(batch)
    return loss

  def forward_on_batch(self, batch):
    xs, ys = batch
    y_hats = self.forward(xs)
    loss = F.binary_cross_entropy(y_hats, ys)
    return loss

  def configure_optimizers(self):
    return torch.optim.SGD(self.parameters(), lr=0.0)

The following code cell shows how to execute your model on the test set
and generate your results as an artifact for submission.

It is based on the code in the section "Run Your Model on the Evaluation Data" of the starter notebooks
([PyTorch Lightning](https://colab.research.google.com/github/wandb/davis-contest/blob/main/colabs/starter_torch.ipynb#scrollTo=y7gj0eF8yc2S),
[Keras](https://wandb.me/davis-starter-keras)),
which ran models on the validation data.

The key difference is that here,
the data being used is from the test set,
rather than the validation set.



Because the data is in the same format
(described in the
[starter notebooks](https://colab.research.google.com/github/wandb/davis-contest/blob/main/colabs/starter_torch.ipynb#scrollTo=aw9pYa8vxEGJ))
as the validation set,
you should be able to use the exact same pipeline you used to submit your results on the validation set to submit your results on the test set.

In particular, once you've downloaded the test data
and constructed a `pd.DataFrame` of paths
(see code snippet below),
the remainder of the evaluation process
should proceed exactly as on the validation data.
```python
with wandb.init(project="davis", job_type="run-test", save_code=True) as run:
  test_data_artifact = run.use_artifact(test_data_artifact_id)
  test_data_paths = paths.artifact_paths(test_data_artifact)
```

As during validation,
the result is a W&B Artifact with a particular structure,
described [in the starter notebooks](https://colab.research.google.com/github/wandb/davis-contest/blob/main/colabs/starter_torch.ipynb#scrollTo=pKYX3uWwPfWW). 



If you run into any issues, reach out to the contest support in any of these ways:
- post in `#qualcomm-competition` on the Weights & Biases [Slack forum](https://wandb.me/slack),
- email `support@wandb.com`, or
- message W&B tech support via the grey speech bubble icon on [wandb.ai](https://wandb.ai) (bottom right-hand corner).

In [ ]:
output_dir = "outputs"
!rm -rf {output_dir}
!mkdir -p {output_dir}

result_artifact_name = "test-result"

with wandb.init(project="davis", job_type="run-test") as run:

  test_data_artifact = run.use_artifact(test_data_artifact_id)
  test_data_paths = paths.artifact_paths(test_data_artifact)

  ####
  ## REPLACE THIS WITH YOUR DATA LOADING CODE, IF NEEDED
  ####
  test_dataset = contest.torch.data.VidSegDataset(
    test_data_paths, has_annotations=False)
  num_images = len(test_dataset)
  test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1)

  ####
  ## REPLACE THIS WITH YOUR MODEL LOADING CODE
  ####
  model_artifact_name = "charlesfrye/davis/dummy-baseline" 
  model_tag = "latest"
  model = contest.torch.utils.load_model_from_artifact(
    model_artifact_name + ":" + model_tag, DummyModel) 

  ## If using Keras, see starter notebook for
  ##   code to print and count params of your model

  # print a summary of the model
  print(model)

  # the number of parameters in the model must be logged
  print("\n")
  nparams = contest.torch.profile.count_params(model)

  profiling_info = {"nparams": nparams}
  wandb.log(profiling_info)

  ####
  ## REPLACE THIS WITH YOUR MODEL EXECUTION CODE, IF NEEDED
  ####
  output_paths = contest.torch.evaluate.run(
    model, test_dataloader, num_images, output_dir)

  # the number of parameters in the model should also be included in the result artifact
  result_artifact = contest.evaluate.make_result_artifact(
    output_paths, result_artifact_name + "-" + run.entity, metadata=profiling_info)
  run.log_artifact(result_artifact)

  run_path = run.path

# Submit to the Final Leaderboard

Once you've logged your results to W&B,
you can officially submit the run to the
final leaderboard on the
[Weights & Biases benchmark for the test dataset](http://wandb.me/davis-test-benchmark).

This works much like submission for results on the validation set,
except that your run doesn't have a `segmentation_metric` score --
that'll be calculated behind the scenes.

To submit, click the "Submit a run" tab on the right side of the 
[benchmark page](https://wandb.me/davis-test-benchmark)
and then paste or type in the
W&B run path for the run
you used to score your submission.
Click the "Submit" button
to start the submission process.

Submissions will be manually reviewed.
Results submitted before the deadline but only reviewed after it
are still eligible.

If you ran the cells above to execute your run, the run path will be printed by the cell below.

See the instructions on the
"Submit a run" tab
for more on run paths.

In [ ]:
run_path